In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
path = '/content/drive/MyDrive/'

In [14]:
import re
import json
import pandas as pd

In [15]:
# 미리 저장된 파일 경로
file_path = path + '텍스트계약서/'+'24년 개정 직매입 표준거래계약서(면세점).txt'
#file_path = path + '텍스트계약서/' + '24년 개정 특약매입 표준거래계약서(면세점).txt'

In [16]:
# 텍스트 파일에서 데이터를 읽고 "제n조" 단위로 분리하고, 빈 문자열을 제거하는 함수
def extract_and_modify_contract(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # "제n조" 단위로 텍스트를 분리
    pattern = r'(제\d+조(?!\S))'  # "제n조" 뒤에 공백이 있거나 끝났을 때
    matches = re.split(pattern, text)

    sentences = {}
    current_title = None
    sentence_number = {}

    for part in matches:
        if part.startswith('제') and '조' in part:
            current_title = re.sub(r'(\d+조)(\[)', r'\1 \2', part.strip())
            number_match = re.match(r'제(\d+)조', current_title)
            if number_match:
                num = number_match.group(1)
                if num in sentence_number:
                    sentence_number[num] += 1
                else:
                    sentence_number[num] = 1
                key = num if sentence_number[num] == 1 else f"{num}_{sentence_number[num]}"
            else:
                key = current_title

            sentences[key] = current_title.strip()
        elif current_title:
            if re.match(r'제\d+조\w', part.strip()):
                sentences[key] += part.strip()
            else:
                sentences[key] += part.strip()

    def split_sentences(text):
    # 먼저 \n\n을 기준으로 분리
      sentences = re.split(r'(\n\n)', text)

    # \n\n을 기준으로 분리한 후, .\n 기준으로 추가 분리
      result = []
      for sentence in sentences:
          if sentence.strip():  # 공백이 아닌 경우에만 처리
            # .\n을 기준으로 분리하고, 리스트에 추가
            result.extend(sentence.split("\n"))

      return result


    # "제n조"와 "제n조의m"을 그룹화하여 처리하는 함수
    def group_content_sections(data):
        grouped_data = {}
        for key, value in data.items():
            content_sentences = split_sentences(value.strip())  # content를 리스트화
            temp_key = key  # 기본 키는 현재 key로 설정 (예: "8")
            temp_content = []
            main_content = []  # "제n조"의 주요 내용을 유지하기 위한 리스트

            for sentence in content_sentences:
                match = re.match(r'제(\d+)조의(\d+) ?\[.*?\]', sentence)  # "제n조의m" 찾기
                if match:
                    # 이전 섹션 내용을 저장
                    if temp_key != key:  # 새로운 "제n조의m"이 시작될 때
                        grouped_data[temp_key] = temp_content
                        temp_content = []
                    # "제n조의m" 키 설정
                    num, sub_num = match.groups()
                    temp_key = f"{num}-{sub_num}"
                    temp_content.append(sentence)
                elif temp_key != key:  # "제n조의m" 안에 포함될 내용
                    temp_content.append(sentence)
                else:  # 기본 "제n조" 내용
                    main_content.append(sentence)

            # 마지막 섹션 저장
            if temp_key != key:
                grouped_data[temp_key] = temp_content
            grouped_data[key] = main_content  # 기본 "제n조" 내용 저장

        return grouped_data

    # 수정된 데이터 반환
    def modify_contract(data):
        grouped_data = group_content_sections(data)
        modified_data = {}
        for key, content in grouped_data.items():
            title_match = re.search(r'\[(.*?)\]', content[0]) if content else None
            title = title_match.group(1) if title_match else key

            # content[0] 삭제 (첫 번째 항목 제거)
            content = content[1:] if content else []

            modified_data[key] = {"title": title, "content": content}
        return modified_data


      # 정렬된 키에 맞게 데이터 반환
    def sort_grouped_data(grouped_data):
        # 조항 번호에 따라 정렬
        sorted_grouped_data = {}
        # 정렬 기준: 숫자와 텍스트를 모두 고려하여 정렬
        for key in sorted(grouped_data.keys(), key=lambda x: [int(i) if i.isdigit() else i for i in re.split(r'(\d+)', x)]):
          sorted_grouped_data[key] = grouped_data[key]
        return sorted_grouped_data

    # 빈 문자열을 제거하는 함수
    def del_empty_content(output_json):
        for key, value in output_json.items():
            # "content" 리스트가 존재할 경우
            if "content" in value:
                # "content" 리스트에서 빈 문자열 제거
                value["content"] = [item for item in value["content"] if item != ""]
        return output_json



    # 결과 데이터 수정
    modified_data = modify_contract(sentences)
    modified_data = sort_grouped_data(modified_data)
    # 빈 문자열을 제거
    modified_data = del_empty_content(modified_data)

    # 결과 데이터 반환 (JSON 형태)
    return modified_data


# 결과를 JSON 파일로 저장하는 함수
def save_to_json(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

In [17]:
raw_sentence = extract_and_modify_contract(file_path)
# save_to_json(raw_sentence, 'seperate_contract.json')

In [19]:
contents = {
    key: [
        sentence.strip().replace('"갑"', '갑').replace('"을"', '을')
        for sentence in value['content']
    ]
    for key, value in raw_sentence.items() if 'content' in value
}

In [20]:
contents = {key: sentences for key, sentences in contents.items() if sentences}

In [21]:
rows = []
for key, sentences in contents.items():
    combined_sentence = ' '.join(sentences)  # 문장들을 하나로 합치기
    rows.append({'key': key, 'sentence': combined_sentence})

# DataFrame으로 변환
df = pd.DataFrame(rows)

#임시 저장1
df.to_csv('contract_data.csv', index=False, encoding='utf-8-sig')

In [22]:
# 문장을 ①, ②, ③ 단위로 분리하는 함수
def split_sentences(sentence):
    # 특수문자가 포함된 경우에만 분리하도록 처리
    if re.search(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|[①-⑩])', sentence):
        # ①, ②, ③ 등 특수문자를 기준으로 문장 분리 (번호는 제거)
        split_sentences = re.split(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|[①-⑩])', sentence)

        # 번호를 제거하고, 문장만 리스트로 반환
        sentences = [s.strip() for s in split_sentences if s.strip() and not re.match(r'[①-⑩]', s)]

        return sentences
    else:
        # 특수문자가 없는 경우 그대로 한 문장 반환
        return [sentence.strip()]

# 각 행에 대해 문장을 분리하여 새로운 행 생성
split_rows = []
for _, row in df.iterrows():
    sentences = split_sentences(row['sentence'])
    for sentence in sentences:
        if sentence:  # 빈 문자열은 추가하지 않음
            split_rows.append({'key': row['key'], 'sentence': sentence})

# 분리된 문장을 새로운 DataFrame으로 생성
split_df = pd.DataFrame(split_rows)

# key 삭제
split_df = split_df.drop(columns=['key'])

# 임시저장2 - 호 분리 전
split_df.to_csv('contract_data_split.csv', index=False, encoding='utf-8-sig')

In [23]:
# ①, ②, ③ 등 특수문자를 기준으로 문장 분리 함수
def split_special_char_sentences(sentence):
    # ①, ②, ③ 등을 기준으로 문장 분리 (번호는 포함하지 않음)
    if re.search(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|[①-⑩])', sentence):
        split_sentences = re.split(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|[①-⑩])', sentence)
        sentences = [s.strip() for s in split_sentences if s.strip() and not re.match(r'[①-⑩]', s)]
        return sentences
    else:
        return [sentence.strip()]

# 1., 2. 형태의 문장을 리스트로 저장하는 함수
def find_numbered_sentences(sentence):
    # 1., 2., 3. 형태의 문장 찾기
    numbered_sentences = re.findall(r'\d+\..+?(?=\d+\.|$)', sentence)  # '1.', '2.' 등으로 구분된 문장 찾기
    return numbered_sentences

# 1단계: 특수문자 기준으로 문장을 분리한 후, 번호 있는 문장을 리스트로 처리
split_rows = []
for _, row in df.iterrows():
    # ①, ②, ③ 등을 기준으로 문장을 분리
    sentences = split_special_char_sentences(row['sentence'])

    # 2단계: 번호 기반 문장 리스트화
    sub_sentences = []
    for sentence in sentences:
        sub_sentence_list = find_numbered_sentences(sentence)
        sub_sentences.append(sub_sentence_list if sub_sentence_list else None)  # 빈 리스트는 None으로 처리

    # 3단계: 리스트로 분리된 문장을 삭제하고 나머지 문장만 저장
    for i, sentence in enumerate(sentences):
        # 번호 기반 문장을 정규식으로 삭제
        for numbered_sentence in sub_sentences[i] or []:  # None인 경우 빈 리스트로 처리:
            sentence = re.sub(re.escape(numbered_sentence), '', sentence).strip()

        # 남은 문장과 sub_sentence를 결과로 저장
        split_rows.append({'key': row['key'], 'sentence': sentence, 'sub_sentence': sub_sentences[i]})

# 결과 DataFrame으로 생성
split_df = pd.DataFrame(split_rows)
# key 삭제
split_df = split_df.drop(columns=['key'])
# 결과 CSV로 저장
split_df.to_csv('contract_data_with_sub_sentences.csv', index=False, encoding='utf-8-sig')

In [29]:
# # 호 접근 예시
# split_df['sub_sentence'][4][0]

'1. 상대방에게 금품, 향응, 편의 또는 접대를 요구하거나 제공해서는 아니 되며, 위법하거나 부당한 행위를 하지 아니한다. '

### 호 분리 함수 종합

In [41]:
import pandas as pd
import re

def process_contract_data(file_path):
    # 1. 텍스트 파일에서 데이터를 읽고 "제n조" 단위로 분리하고 빈 문자열을 제거하는 작업
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # "제n조" 단위로 텍스트를 분리
    pattern = r'(제\d+조(?!\S))'  # "제n조" 뒤에 공백이 있거나 끝났을 때
    matches = re.split(pattern, text)

    sentences = {}
    current_title = None
    sentence_number = {}

    for part in matches:
        if part.startswith('제') and '조' in part:
            current_title = re.sub(r'(\d+조)(\[)', r'\1 \2', part.strip())
            number_match = re.match(r'제(\d+)조', current_title)
            if number_match:
                num = number_match.group(1)
                if num in sentence_number:
                    sentence_number[num] += 1
                else:
                    sentence_number[num] = 1
                key = num if sentence_number[num] == 1 else f"{num}_{sentence_number[num]}"
            else:
                key = current_title

            sentences[key] = current_title.strip()
        elif current_title:
            if re.match(r'제\d+조\w', part.strip()):
                sentences[key] += part.strip()
            else:
                sentences[key] += part.strip()

    # 먼저 \n\n을 기준으로 분리
    def split_sentences(text):
        sentences = re.split(r'(\n\n)', text)

        result = []
        for sentence in sentences:
            if sentence.strip():  # 공백이 아닌 경우에만 처리
                result.extend(sentence.split("\n"))

        return result

    # "제n조"와 "제n조의m"을 그룹화하여 처리하는 함수
    def group_content_sections(data):
        grouped_data = {}
        for key, value in data.items():
            content_sentences = split_sentences(value.strip())  # content를 리스트화
            temp_key = key  # 기본 키는 현재 key로 설정 (예: "8")
            temp_content = []
            main_content = []  # "제n조"의 주요 내용을 유지하기 위한 리스트

            for sentence in content_sentences:
                match = re.match(r'제(\d+)조의(\d+) ?\[.*?\]', sentence)  # "제n조의m" 찾기
                if match:
                    # 이전 섹션 내용을 저장
                    if temp_key != key:  # 새로운 "제n조의m"이 시작될 때
                        grouped_data[temp_key] = temp_content
                        temp_content = []
                    # "제n조의m" 키 설정
                    num, sub_num = match.groups()
                    temp_key = f"{num}-{sub_num}"
                    temp_content.append(sentence)
                elif temp_key != key:  # "제n조의m" 안에 포함될 내용
                    temp_content.append(sentence)
                else:  # 기본 "제n조" 내용
                    main_content.append(sentence)

            # 마지막 섹션 저장
            if temp_key != key:
                grouped_data[temp_key] = temp_content
            grouped_data[key] = main_content  # 기본 "제n조" 내용 저장

        return grouped_data

    # 수정된 데이터 반환
    def modify_contract(data):
        grouped_data = group_content_sections(data)
        modified_data = {}
        for key, content in grouped_data.items():
            title_match = re.search(r'\[(.*?)\]', content[0]) if content else None
            title = title_match.group(1) if title_match else key

            # content[0] 삭제 (첫 번째 항목 제거)
            content = content[1:] if content else []

            modified_data[key] = {"title": title, "content": content}
        return modified_data

    # 정렬된 키에 맞게 데이터 반환
    def sort_grouped_data(grouped_data):
        # 조항 번호에 따라 정렬
        sorted_grouped_data = {}
        # 정렬 기준: 숫자와 텍스트를 모두 고려하여 정렬
        for key in sorted(grouped_data.keys(), key=lambda x: [int(i) if i.isdigit() else i for i in re.split(r'(\d+)', x)]):
            sorted_grouped_data[key] = grouped_data[key]
        return sorted_grouped_data

    # 빈 문자열을 제거하는 함수
    def del_empty_content(output_json):
        for key, value in output_json.items():
            # "content" 리스트가 존재할 경우
            if "content" in value:
                # "content" 리스트에서 빈 문자열 제거
                value["content"] = [item for item in value["content"] if item != ""]
        return output_json

    # 결과 데이터 수정
    modified_data = modify_contract(sentences)
    modified_data = sort_grouped_data(modified_data)
    # 빈 문자열을 제거
    modified_data = del_empty_content(modified_data)

    # 2. content 내용 수정 및 저장
    contents = {
        key: [
            sentence.strip().replace('"갑"', '갑').replace('"을"', '을')
            for sentence in value['content']
        ]
        for key, value in modified_data.items() if 'content' in value
    }
    contents = {key: sentences for key, sentences in contents.items() if sentences}

    # 3. DataFrame으로 변환
    rows = []
    for key, sentences in contents.items():
        combined_sentence = ' '.join(sentences)  # 문장들을 하나로 합치기
        rows.append({'key': key, 'sentence': combined_sentence})

    # DataFrame으로 변환
    df = pd.DataFrame(rows)

    # 4. 특수문자 기준으로 문장을 분리하는 함수
    def split_special_char_sentences(sentence):
        if re.search(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|[①-⑩])', sentence):
            split_sentences = re.split(r'(①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|[①-⑩])', sentence)
            sentences = [s.strip() for s in split_sentences if s.strip() and not re.match(r'[①-⑩]', s)]
            return sentences
        else:
            return [sentence.strip()]

    # 5. 번호가 포함된 문장을 찾는 함수
    def find_numbered_sentences(sentence):
        return re.findall(r'\d+\..+?(?=\d+\.|$)', sentence)  # '1.', '2.' 등으로 구분된 문장 찾기

    # 6. 특수문자 기준으로 문장을 분리한 후 번호 있는 문장을 리스트로 처리
    split_rows = []
    for _, row in df.iterrows():
        # ①, ②, ③ 등을 기준으로 문장을 분리
        sentences = split_special_char_sentences(row['sentence'])

        # 번호 기반 문장 리스트화
        sub_sentences = []
        for sentence in sentences:
            sub_sentence_list = find_numbered_sentences(sentence)
            sub_sentences.append(sub_sentence_list if sub_sentence_list else None)  # 빈 리스트는 None으로 처리

        # 리스트로 분리된 문장을 삭제하고 나머지 문장만 저장
        for i, sentence in enumerate(sentences):
            for numbered_sentence in sub_sentences[i] or []:  # None인 경우 빈 리스트로 처리:
                sentence = re.sub(re.escape(numbered_sentence), '', sentence).strip()

            split_rows.append({'key': row['key'], 'sentence': sentence, 'sub_sentence': sub_sentences[i]})

    # 7. 결과 DataFrame 생성
    split_df = pd.DataFrame(split_rows)

    # 8. key 컬럼 삭제
    split_df = split_df.drop(columns=['key'])

    # 9. 결과 CSV로 저장
    split_df.to_csv('contract_data_with_sub_sentences.csv', index=False, encoding='utf-8-sig')

    return split_df

In [44]:
# 미리 저장된 파일 경로
#file_path = path + '텍스트계약서/'+'24년 개정 직매입 표준거래계약서(면세점).txt'
file_path = path + '텍스트계약서/' + '24년 개정 특약매입 표준거래계약서(면세점).txt'
# process_contract_data 함수를 실행하여 계약서 처리
result_df = process_contract_data(file_path)